In [123]:
import nltk
import numpy as np
import random
import string

In [124]:
f=open('/Users/amipatel/Desktop/kb.txt','r',errors = 'ignore')
raw=f.read()
raw=raw.lower()
raw=' '.join(raw.split())

sent_tokens = nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

In [125]:
sent_tokens[:2]

['people tend to lose weight quickly on low carb diets because they restrict their calories to about 1,200 a day.',
 "this is about 500 calories per day for women, and 600 for men.you can choose whichever two days of the week you prefer, as long as there is at least one non fasting day in between them.one common way of planning the week is to fast on mondays and thursdays, with two or three small meals, then eat normally for the rest of the week.it's important to emphasize that eating normally does not mean you can eat anything."]

In [126]:
inputs = ("hello", "hi", "greetings", "what up", "what's up","hey",)
response1 = ["hi", "hey", "hi there", "hello", "hi, I'm good"]

def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in inputs:
            return random.choice(response1)

In [127]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [128]:
from textblob import TextBlob
import re

In [129]:
questionTriples = ['CD-VB-VBN',
                   'MD-PRP-VB' ,
                   'MD-VB-CD' ,
                   'NN-IN-DT' ,
                   'PRP-VB-PRP' ,
                   'PRP-WP-NNP' ,
                   'VB-CD-VB' ,
                   'VB-PRP-WP' ,
                   'VBZ-DT-NN' ,
                   'WP-VBZ-DT' ,
                   'WP-VBZ-NNP' ,
                   'WRB-MD-VB']

statementTriples = ['DT-JJ-NN',
                   'DT-NN-VBZ',
                   'DT-NNP-NNP',
                   'IN-DT-NN',
                   'IN-NN-NNS',
                   'MD-VB-VBN',
                   'NNP-IN-NNP',
                   'NNP-NNP-NNP',
                   'NNP-VBZ-DT',
                   'NNP-VBZ-NNP',
                   'NNS-IN-DT',
                   'VB-VBN-IN',
                   'VBZ-DT-JJ']

simpleChat = [['PRP','VBP', 'DT', 'JJ', 'NN'], ['PRP','VBP','DT', 'NN'], ['PRP','VBP', 'JJ'], ['PRP','VBP', 'JJ', 'NN']] 
def find_prp(s):
    response = None
    
    for w,POS in s.pos_tags:
        if w == 'you':
            response = 'I'
        if w == 'i':
            response = 'You'
            
    return response

def find_nn(s):
    nn = None
    
    for w,POS in s.pos_tags:
        if 'NN' in POS:
            nn = w
            
    return nn
    
def find_vp(s):
    vb = None
    
    for w,POS in s.pos_tags:
        if 'VB' in POS:
            if w == 'am':
                vb = 'are'
            if w == 'are':
                vb = 'am'
            else:
                vb = w
            
    return vb

def find_adj(s):
    adj = None
    
    for w,POS in s.pos_tags:
        if 'JJ' in POS:
            adj = w
            
    return adj

def find_dt(s):
    dt = None
    
    for w,POS in s.pos_tags:
        if 'DT' in POS:
            dt = w
            
    return dt

def find_parts_of_speech(parsed):
    for s in parsed.sentences:
        prp_response = find_prp(s)
        nn_response = find_nn(s)
        vb_response = find_vp(s)
        adj_response = find_adj(s)
        dt_response = find_dt(s)
    return prp_response, nn_response, vb_response, adj_response, dt_response
    
def response2(user):
    sent_tokens.append(user)
    user = user.lower()
    punctuations = '''!()-[]{};:"\,<>/@#$%^&*_~'''
    for x in user: 
        if x in punctuations: 
            user = user.replace(x, " ") 
    content = re.sub(r'[^a-z0-9 .?]+', ' ',user)
    parsed = TextBlob(content)
    prp, nn, vb, adj, dt = find_parts_of_speech(parsed)
    pos = []
    for w,POS in parsed.pos_tags:
        pos.append(POS)
    res = ''
    if pos in simpleChat:
        res = 'Sure, '
        if parsed.noun_phrases is not []:
            if nn is not None:
                if dt is None and adj is not None:
                    res += prp + ' ' + vb + ' ' + adj + ' ' + nn + '!'
                elif dt is None and adj is None:
                    res += prp + ' ' + vb + ' ' + nn + '!'
                elif dt is not None and adj is None:
                    res += prp + ' ' + vb + ' ' + dt + ' ' + nn + '!'
                else:
                    res += prp + ' ' + vb + ' ' + dt + ' ' + adj + ' ' + nn + '!'
            else:
                if dt is None and adj is not None:
                    res += prp + ' ' + vb + ' ' + adj + '!'
                elif dt is None and adj is None:
                    res += prp + ' ' + vb + '!'
                elif dt is not None and adj is None:
                    res += prp + ' ' + vb + ' ' + dt + '!'
                else:
                    res += prp + ' ' + vb + ' ' + dt + ' ' + adj + '!'
        elif adj is not None:
            res += prp + ' ' + vb + ' ' + adj + '!'
        else:
            if dt is not None and nn is not None:
                res += prp + ' ' + vb + ' ' + dt + ' ' + nn + '!'
        sent_tokens.remove(user)
        return res
    else:
        return response(user)
    

In [130]:
def response(user_response):
    bot_response=''
    user_response = user_response.lower()
    punctuations = '''!()-[]{};:"\,<>/@#$%^&*_~'''
    for x in user_response: 
        if x in punctuations: 
            user_response = user_response.replace(x, " ") 
    content = re.sub(r'[^a-z0-9 .?]+', ' ',user_response)
    question = False
    use = ''    
    if '?' in content:
        if(content=='how are you?' or content=='what is your name?'):  
            if content=='how are you?':
                bot_response = 'I am fine!'
            else:
                bot_response = 'My name is DIETO. I will answer your queries about diets.'
            sent_tokens.remove(user_response)
            return bot_response
        elif 'diet' in content:
            parse = TextBlob(content)
            if ('what', 'WP') in parse.pos_tags:
                question = True
                for w,POS in parse.pos_tags:
                    if 'WP' in POS or 'VB' in POS:
                        continue
                    else:
                        if w is not 'diet':
                            use += w
                        elif POS == 'JJ':
                            use += w
                        
            sent_tokens.append(use)   
   
    TfidfVec = TfidfVectorizer()
    tfidf = TfidfVec.fit_transform(sent_tokens)
    similar_vector_vals = cosine_similarity(tfidf[-1], tfidf)
    
    # the second last sentence will have the highest cosine similarity
    if question is True:
        sentence_number=similar_vector_vals.argsort()[0][-3]  
    else:
        sentence_number=similar_vector_vals.argsort()[0][-2]
    
    # Flatten to check if the cosine similarity is 0 or not
    flat = similar_vector_vals.flatten()
    flat.sort()
    actual_tfidf = flat[-2]
    if(actual_tfidf==0):
        bot_response=bot_response+"I am sorry! I don't understand you"
        sent_tokens.remove(user_response)
        return bot_response
    else:
        bot_response = bot_response+sent_tokens[sentence_number]
        if question is True:
            sent_tokens.remove(use)
        sent_tokens.remove(user_response)
        return bot_response

In [131]:
Bye = False
print("DIETO: My name is DIETO. I will answer your queries about diets. If you want to exit, type Bye!")
while(Bye is not True):
    print("YOU: ", end = '')
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            print("DIETO: You are welcome.")
        else:
            if(greeting(user_response)!=None):
                print("DIETO: "+greeting(user_response))
            else:
                print("DIETO: ",end="")
                print(response2(user_response))
    else:
        Bye = True
        print("DIETO: Bye! take care..")

DIETO: My name is DIETO. I will answer your queries about diets. If you want to exit, type Bye!
YOU: hi
DIETO: hi, I'm good
YOU: how are you?
DIETO: I am fine!
YOU: how many calories should a normal person consume?
DIETO: experts recommend that healthy adult women consume about 2,000 calories and man consume 2,500 calories daily.
YOU: what is diet?
DIETO: low fat diet, low glycemic index diet, low protein diet, low sodium diet, low sulfur diet, some common macrobiotic ingredients macrobiotic diet: a diet in which processed food is avoided.
YOU: how much is the normal weight?
DIETO: mazel states that it is not what or how much you eat that causes weight gain but rather when and in what combinations the food is eaten.
YOU: how much healthy is the vegetarian food?
DIETO: how they work with these diets, ketosis is the key to losing weight but not to healthy nutrition.
YOU: what is ketosis?
DIETO: the more serious question is what even brief exposure to ketosis does to the body long term.
Y

In [132]:
text = input()
p = TextBlob(text)
print(p.pos_tags)
p.noun_phrases


what is diet?
[('what', 'WP'), ('is', 'VBZ'), ('diet', 'JJ')]


WordList([])